# Проверка качества

In [14]:
import os
import sys
import time
import json
import pickle
import warnings

from collections import defaultdict

sys.path.append('../')
sys.path.append('old_configs/')
warnings.filterwarnings('ignore')
try:
    sys.modules.pop('src.modeling.autocompletion')
except:
    print('cant do')

import torch
import tokenizers
import numpy as np
import pandas as pd
import torch.nn as nn
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
from pygments.token import Token
from catalyst.utils import load_checkpoint, unpack_checkpoint

from src.preprocessing.preprocessing import LexerBasedPreprocessor
from src.preprocessing.tokenization import BertWordPieceTokenizerWrapper, SentencepieceTokenizerWrapper
from src.generation.generation_utils import TokenScoresPostprocessor, NextTokenChooser
from src.generation.autocompletion import AutocompletionModel
from src.generation.postprocessing import get_first_word
from src.preprocessing.tokenization import clever_join
from src.generation.prefix_utils import PrefixMatcher
from src.utils.metrics import reciprocal_rank, relevant_in_k

from run_gpt_ddp_training import GPT2ConfigInitializer

cant do


In [15]:
CUDA = 0

In [16]:
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [17]:
# preprocesser
DATA_DIR = '/mnt/data/popov/rcompletion/evaluation_data/'
with open('/home/popov/data/rcompletion_files/december2020_best_model/top_tokens_bigdata_021020.json', 'r') as f:
    top_tokens = json.load(f)
top_tokens = set(top_tokens)
preprocesser = LexerBasedPreprocessor(protected_names=top_tokens)
lexer = preprocesser.lexer
# tokenizer
tokenizer = SentencepieceTokenizerWrapper(f'/mnt/data/porkhun/tokenizer/spm_cased_bpe_16.model')

In [18]:
from gpt_config import Config as gpt_config

initializer = GPT2ConfigInitializer(gpt_config)
model = initializer.init_model()
checkpoint = load_checkpoint('/home/porkhun/model_training/practice/const_shift/shift25/best.pth')
unpack_checkpoint(checkpoint=checkpoint, model=model)

In [19]:
# model = model.student_model
model = model.eval().to(f'cuda:{CUDA}')

In [20]:
def create_df(directory):
    dict_for_df = defaultdict(list)
    with open(directory, 'r') as f:
        for line in f:
            d = json.loads(line)
            for key in d:
                dict_for_df[key].append(d[key])
    return pd.DataFrame(dict_for_df)

In [21]:
if hasattr(model, 'double_context'):
    df = create_df(DATA_DIR+'extracted_events_with_right_context.json')
else:
    df = create_df(DATA_DIR+'extracted_events.json')
df.head()

,url,before_cursor,after_cursor,after_cursor_token,right_context,group,prefix
0,https://api.github.com/repos/CenterForStatisti...,########################\n# ...,otlist,plotlist,"=plotlist,labels=LETTERS[1:4])\ndev.off()",f_key_argument,prefix
1,https://api.github.com/repos/VUW-FAIR/tic-pers...,"setwd(""/Users/mlr/OneDrive - Victoria Universi...",thod,method,"= ""euclid"")\nbarplot(unlist(allDistances))\n\...",f_key_argument,prefix
2,https://api.github.com/repos/jayhesselberth/gg...,"context(""ggsave"")\n\ntest_that(""ggsave creates...",cale,scale,"= 2), c(10, 10))\n})\n\n\n# plot_dev --------...",f_key_argument,prefix
3,https://api.github.com/repos/gtesei/fast-furio...,require(xgboost)\nrequire(methods)\nlibrary(da...,op,drop,"= F]) \n test_set = cbind(test_set , data[...",f_key_argument,prefix
4,https://api.github.com/repos/HadiEO/tropical_r...,require(ggplot2)\nrequire(tidyverse)\nrequire(...,ir,nir,"=seq(0,100,20), swir1=seq(0,100,20), swir2=seq...",f_key_argument,prefix


Для измерения зависимости от итерации:

In [ ]:
%%time
all_real_outputs = dict()
all_model_outputs = dict()

lines_to_keep = 100
# for lines_to_keep in [None, 85, 70, 80, 100, 140]:
autocompletion_model = AutocompletionModel(
        preprocessor=preprocesser,
        tokenizer=tokenizer,
        model=model,
        score_postprocesser=TokenScoresPostprocessor(temperature=1.5, penalty_theta=0.5),
        next_token_chooser=NextTokenChooser(do_sample=False),
        max_tokens_amount=5,
        num_beams=5,
        max_num_sequence_return=20,
        input_lines_to_keep=lines_to_keep,
    )
    
model_outputs = []
real_outputs = []
bad_indexes = []
for i, elem in tqdm(df.iterrows()):
    if autocompletion_model.double_context:
        test_sample = (elem['before_cursor'], elem['right_context'])
        
    else:
        test_sample = elem['before_cursor']
    try:
        one_model_outputs = autocompletion_model.autocomplete_input(
            test_sample,
            drop_last_word='always' if elem['prefix'] == 'prefix' else 'never',
        )
        one_real_output = elem['after_cursor_token']
        model_outputs.append(one_model_outputs)
        real_outputs.append(one_real_output)
    except Exception:
#         raise
        bad_indexes.append(i)
#     if i > 10:
#         break

In [ ]:
# delete all brackets from left context
real_o = real_outputs
model_o = model_outputs
relevances = [
    [int(x == one_r_o) for x in one_model_o]
    for one_r_o, one_model_o in zip(real_o, model_o)
]
key_metrics = [
    [relevant_in_k(one_r, k=k) for k in range(1, 6)] + [reciprocal_rank(one_r)]
    if one_r else [0] * 6
    for one_r in relevances
]
key_metrics = np.array(key_metrics).mean(axis=0)
key_metrics

In [ ]:
## BiGPT 25 shift

In [ ]:
pd.DataFrame([key_metrics], columns=['Top 1', 'Top 2', 'Top 3', 'Top 4', 'Top 5', 'MRR']).round(3)

In [ ]:
len(bad_indexes)